In [137]:
from test_utils import * 
import space_transmit
import datetime
import pandas as pd 
from pandas.util.testing import assert_frame_equal
import numpy as np

from influxdb import DataFrameClient
from influxdb import InfluxDBClient

import json

In [121]:
def process_data(raw_data,units):
    df=pd.DataFrame.from_records(raw_data)
    df['datetime']=pd.to_datetime(pd.DatetimeIndex(df.datetime))
    df.value=pd.to_numeric(df.value,errors='coerce').astype('float')
    interval=round(pd.Series(df.datetime.unique()).diff().dropna().mean()/np.timedelta64(1, 'm'))
    df=df.set_index('datetime')
    if interval==60.0 and (units=='kW' or units=='kWh'):
        df= disaggregate(df)
    elif interval== 5.0 and units=='kW':
        df=df*(5/60)
        df=df.resample('15T').sum()
    elif interval == 5.0 and units =='kWh':
        df=df.resample('15T').sum()
    elif interval ==15.0 and units=='kW':
        df=df/4
    elif interval==15.0 and units=='kWh':
        df=df
    else:
        return(Exception)
    return df

def disaggregate(df):
    df=df/4
    df=df.resample('15T').mean().interpolate(method='pad',axis=1,limit=3,limit_direction='forward')
    ds=pd.date_range(start=df.index[-1]+datetime.timedelta(minutes=15),periods=3,freq='15T')
    ve=[df.iloc[-1].value]*3
    new_df=pd.DataFrame({'datetime':ds,'value':ve})
    new_df=new_df.set_index('datetime')
    df=df.append(new_df)
    return df

In [135]:
def query_db(s_id,start_date,end_date):
    client=InfluxDBClient(host='127.0.0.1',port=8086,database='space_app')
    sd=format_date(start_date)
    ed=format_date(end_date)
    q="SELECT * FROM autogen.power where (site_id='{}' AND time >='{}' AND time <='{}')".format(s_id,sd,ed)
    x=client.query(q,database='space')
    x_dict=x.raw
    x_df=pd.DataFrame.from_records(x_dict.get('series')[0].get('values'),columns=x_dict.get('series')[0].get('columns'))
    x_df=x_df.rename(columns={'time':'datetime'})
    data=x_df[['datetime','value']].to_json(orient='records')
    frame={"spaceship_id":x_df.site_id.iloc[0],"units":x_df.units.iloc[0],"data":json.loads(data)}
    return frame

def format_date(sd):
    return sd.replace('-',':').replace(':','-',2)

In [128]:
raw_data=pd.read_pickle('./raw_data_5M.pickle')
#processed_df=pd.read_pickle('./processed_df_1H.pickle')
units='kW'
space_transmit.process_data(raw_data,units)

,value
datetime,
2018-08-24 00:00:00+00:00,5.750000
2018-08-24 00:15:00+00:00,5.250000
2018-08-24 00:30:00+00:00,0.666667


In [118]:
raw_data

[{'datetime': '2018-08-24T00:00:00Z', 'value': 12},
 {'datetime': '2018-08-24T01:00:00Z', 'value': 26},
 {'datetime': '2018-08-24T02:00:00Z', 'value': 18}]

In [129]:
processed_df=process_data(raw_data,units)

In [130]:
processed_df.to_pickle('./processed_df_5M.pickle')

In [144]:
pd.to_pickle(query_db('11','2018-08-24T00-00-00Z','2018-08-24T03-00-00Z'),'./query_db_result.pickle')